## Load Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%cp '/content/gdrive/MyDrive/datasets/ML Dataset/ml_project_dataset.pkl' '/content/'

In [ ]:
!pip install pickle5

In [ ]:
import pickle5 as pickle
import numpy as np

with open('ml_project_dataset.pkl', "rb") as fh:
  data = pickle.load(fh)

## Show Dataset

In [ ]:
data.head()

,Age,Gender,Status,f1,f2,f3,f4,f5
0,28,Female,Happy,"[-0.03723609, -0.020824267, -0.025246128, 0.00...","[-0.0013615170028060675, -0.000550377124454826...","[-0.045345112681388855, 0.049214184284210205, ...","[0.622996985912323, 0.5587776899337769, 0.2035...","[142.0, 268.0, -91.8211441040039, 147.0, 304.0..."
1,24,Male,Sad,"[-0.009315872, -0.0008500488, 0.0008058883, 0....","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[0.22100389003753662, 0.0, 0.0, 0.070420034229...","[1302.0, 2091.0, -218.44000244140625, 928.0, 2..."
2,29,M,neutral,"[-0.013662366, 0.03466877, -0.065040134, -0.04...","[-0.008576362393796444, 0.09807106107473373, -...","[-0.002319525694474578, 0.03656185418367386, -...","[0.0, 0.21522775292396545, 0.26262786984443665...","[831.0, 252.0, -65.05692291259766, 909.0, 516...."
3,28,M,happy,"[-0.06278418, -0.03465886, -0.035757925, -0.02...","[0.03950166329741478, 0.1211700439453125, 0.03...","[-0.02030731551349163, 0.0438639298081398, -0....","[0.021185416728258133, 0.015900805592536926, 0...","[796.0, 715.0, -420.5904541015625, 850.0, 845...."
4,25,Male,Normal,"[0.027600960806012154, 0.03169381991028786, -0...","[-0.012523038312792778, 0.1949128955602646, 0....","[-0.024486687034368515, 0.031889624893665314, ...","[0.36306896805763245, 0.45892614126205444, 0.1...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."


In [ ]:
data.Gender.unique()

array(['Female', 'Male', 'M', 'female', 'male', 'F', 'm'], dtype=object)

In [ ]:
data.Status.unique()

array(['Happy', 'Sad', 'neutral', 'happy', 'Normal', 'sad', '604e0ee7bf',
       'natural', 'Neutral', 'H', '0d02ffb6f5', 'S', '0d02ffb6f8',
       '608e0ee7bf', 'happy ', '601e0ee7bf', '0042a40fde', 'N',
       '0049a40fde', '602e0ee7bf', '77ae9e7a46', '607e0ee7bf',
       '0047a40fde', '603e0ee7bf', '609e0ee7bf', '77ae9e7a41',
       '0d02ffb6f15', '0d02ffb6f9', '0040a40fde', '77ae9e7a51',
       '0050a40fde', '77ae9e7a43', '77ae9e7a39', '0d02ffb6f4',
       '0051a40fde', '0046a40fde', '0d02ffb6f14', '0043a40fde',
       '0054a40fde', '77ae9e7a44', '606e0ee7bf', '77ae9e7a45',
       '610e0ee7bf', '0d02ffb6f11', '0d02ffb6f3', '611e0ee7bf',
       '77ae9e7a38', '0052a40fde', '0d02ffb6f1', '77ae9e7a50',
       '597e0ee7bf', '598e0ee7bf', '0044a40fde', '0d02ffb6f7',
       '0d02ffb6f6', '0041a40fde', '77ae9e7a48', '77ae9e7a47',
       '0d02ffb6f12', '0048a40fde', '0d02ffb6f2', '600e0ee7bf',
       '605e0ee7bf', '77ae9e7a40', '0053a40fde', '599e0ee7bf',
       '77ae9e7a42', '77ae9e7a49', 

## Preprocess for categorical data

In [ ]:
import re

data.Gender=data.Gender.apply(lambda x: re.sub('^[fF]\w*','f',x))
data.Gender=data.Gender.apply(lambda x: re.sub('^[mM]\w*','m',x))

data.Status = data.Status.apply(lambda x : re.sub('^[sS]+.*','s',x))
data.Status = data.Status.apply(lambda x : re.sub('^[hH]+.*','h',x))
data.Status = data.Status.apply(lambda x : re.sub('^[nN]+.*','n',x))

#find cropted label and drop the row data thats containe it 
data.Status = data.Status.apply(lambda x : re.sub('^\d+.*','cropted',x))
data.Status.replace('cropted',np.nan,inplace=True)
data.dropna(subset=['Status'],inplace=True)

data.Gender=data.Gender.astype('category').cat.codes
data.Status=data.Status.astype('category').cat.codes

In [ ]:
data_f1 = data[['Age','Gender','Status','f1']].copy()
data_f2 = data[['Age','Gender','Status','f2']].copy()
data_f3 = data[['Age','Gender','Status','f3']].copy()
data_f4 = data[['Age','Gender','Status','f4']].copy()
data_f5 = data[['Age','Gender','Status','f5']].copy()

## Clean NaN data

In [ ]:
data_f1.f1=data_f1.f1.apply(lambda x: np.nan if np.isnan(x[0]) else x)
data_f1.dropna(subset=['f1'],inplace=True)
data_f2.f2=data_f2.f2.apply(lambda x: np.nan if np.isnan(x[0]) else x)
data_f2.dropna(subset=['f2'],inplace=True)
data_f3.f3=data_f3.f3.apply(lambda x: np.nan if np.isnan(x[0]) else x)
data_f3.dropna(subset=['f3'],inplace=True)
data_f4.f4=data_f4.f4.apply(lambda x: np.nan if np.isnan(x[0]) else x)
data_f4.dropna(subset=['f4'],inplace=True)
data_f5.f5=data_f5.f5.apply(lambda x: np.nan if np.isnan(x[0]) else x)
data_f5.dropna(subset=['f5'],inplace=True)

data_f1.reset_index(drop=True,inplace=True)
data_f2.reset_index(drop=True,inplace=True)
data_f3.reset_index(drop=True,inplace=True)
data_f4.reset_index(drop=True,inplace=True)
data_f5.reset_index(drop=True,inplace=True)

In [ ]:
!pip install -U imbalanced-learn

     |████████████████████████████████| 215kB 6.8MB/s 
     |████████████████████████████████| 22.3MB 123kB/s 
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1
  Found existing installation: imbalanced-learn 0.4.3
    Uninstalling imbalanced-learn-0.4.3:
      Successfully uninstalled imbalanced-learn-0.4.3


In [ ]:
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from warnings import filterwarnings
filterwarnings('ignore')

def Dimension_Reduction(df,y_target):
  regressor = LinearRegression()
  regressor.fit(df,y_target)
  intercept = regressor.intercept_
  features = pd.DataFrame(regressor.coef_,df.columns,columns=['coefficient'])
  features.coefficient = features.coefficient.abs()
  stdvs=[]
  for i in df.columns:
    stdv=df[i].std()
    stdvs.append(stdv)
  features['stdev']=np.array(stdvs).reshape(-1,1)
  features['importance']=features['coefficient']*features['stdev']
  features['importance_normalized'] = 100*features['importance'] / features['importance'].max()
  xxx=features.importance_normalized
  indx=xxx[xxx>15].index
  x_sell=df[indx]

  cols = list(x_sell.columns)
  del_cols=list()
  pmax = 1
  count=0
  while (len(cols)>0):
      count+=1
      p= []
      X_1 = x_sell[cols]
      X_1 = sm.add_constant(X_1)
      model = sm.OLS(y_target,X_1).fit()
      p = pd.Series(model.pvalues.values[1:],index = cols)      
      pmax = max(p)
      feature_with_p_max = p.idxmax()
      if(pmax>0.001):
          del_cols.append(feature_with_p_max)
          cols.remove(feature_with_p_max)
      else:
          break

  selected_features_BE = cols
  print('>> The input dimension is',df.shape[1],'and it is reduced to',len(selected_features_BE))
  return x_sell[selected_features_BE].values.copy()

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LinearRegression,LogisticRegression
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures, StandardScaler, RobustScaler,MinMaxScaler
from sklearn.pipeline import Pipeline
from matplotlib import pyplot as plt 
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE
from sklearn.svm import SVC
from collections import Counter
import pandas as pd
import statsmodels.api as sm
from time import time
from prettytable import PrettyTable as pt
from warnings import filterwarnings
filterwarnings('ignore')

def k_with_tuninig(train_data,test_data,train_target,test_target):
      leaf_size = list(range(1, 10))
      n_neighbors = list(range(3, 20))
      p = [1, 2]
      knn = KNeighborsClassifier()
      hyperparameters = dict(leaf_size=leaf_size, n_neighbors=n_neighbors, p=p)
      clf = GridSearchCV(knn, hyperparameters, cv=10)
      best_model = clf.fit(train_data, train_target.ravel())# Best Hyperparameters Value
      print('Best leaf_size:', best_model.best_estimator_.get_params()['leaf_size'])
      print('Best p:', best_model.best_estimator_.get_params()['p'])
      print('Best n_neighbors:', best_model.best_estimator_.get_params()['n_neighbors'])  
      predict_target_test = best_model.predict(test_data)
      predict_target_train = best_model.predict(train_data)
      print(classification_report(test_target, predict_target_test))

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

def make_model_classification_knn(X,feature_set,y_target,use_LDA=True):
  x=X[feature_set].values.copy()
  u = np.zeros((x.shape[0],x[0].shape[0]))
  for i in range(x.shape[0]):
    for j in range(x[0].shape[0]):
      u[i][j]=x[i][j]
    
  cols = np.arange(0,x[0].shape[0],1)
  df = pd.DataFrame(u, columns = cols)
  oversample = SMOTE()
  x_oversample, y_oversample_target_fs = oversample.fit_resample(df,y_target.values.reshape(-1,1))
  print(Counter(y_oversample_target_fs))
  x_reduction=Dimension_Reduction(x_oversample,y_oversample_target_fs)
  
  if use_LDA:
    clf = LinearDiscriminantAnalysis()
    x_reduction = clf.fit_transform(x_reduction,y_oversample_target_fs)
  SS = StandardScaler()
  for i in range(x_reduction.shape[1]):
    x_reduction[:,i]=SS.fit_transform(x_reduction[:,i].reshape(-1,1)).reshape(-1,)

  x_train , x_test , y_train , y_test = train_test_split(x_reduction,y_oversample_target_fs, test_size=0.33,random_state=47)
  k_with_tuninig(x_train , x_test , y_train , y_test)

def make_model_classification(X,feature_set,y_target,use_LDA=True):
  x=X[feature_set].values.copy()
  u = np.zeros((x.shape[0],x[0].shape[0]))
  for i in range(x.shape[0]):
    for j in range(x[0].shape[0]):
      u[i][j]=x[i][j]
    
  cols = np.arange(0,x[0].shape[0],1)
  df = pd.DataFrame(u, columns = cols)
  
  oversample = SMOTE()
  x_oversample, y_oversample_target_fs = oversample.fit_resample(df,y_target.values.reshape(-1,1))
  print(Counter(y_oversample_target_fs))
  
  x_reduction=Dimension_Reduction(x_oversample,y_oversample_target_fs)
  
  
  if use_LDA:
    clf = LinearDiscriminantAnalysis()
    x_reduction = clf.fit_transform(x_reduction,y_oversample_target_fs)

  SS = StandardScaler()
  for i in range(x_reduction.shape[1]):
    x_reduction[:,i]=SS.fit_transform(x_reduction[:,i].reshape(-1,1)).reshape(-1,)
  x_train , x_test , y_train , y_test = train_test_split(x_reduction,y_oversample_target_fs, test_size=0.33,random_state=47)
  svc_target_fs = SVC()
  svc_target_fs.fit(x_train,y_train)
  pred_target_fs = svc_target_fs.predict(x_test)
  print(classification_report(y_test,pred_target_fs))


def make_model_regression(X,feature_set,y_target):
  x=X[feature_set].values.copy()
  u = np.zeros((x.shape[0],x[0].shape[0]))
  for i in range(x.shape[0]):
    for j in range(x[0].shape[0]):
      u[i][j]=x[i][j]
    
  cols = np.arange(0,x[0].shape[0],1)
  df = pd.DataFrame(u, columns = cols)
 
  oversample = SMOTE(k_neighbors=2)
  x_oversample, y_oversample_target_fs = oversample.fit_resample(df,y_target.values.reshape(-1,1))
  x_reduction=Dimension_Reduction(x_oversample,y_oversample_target_fs)

  x_train , x_test , y_train , y_test = train_test_split(x_reduction,y_oversample_target_fs, test_size=0.33,random_state=7)
  
  pipe = Pipeline([
  ('sc',StandardScaler()),
  ('poly',PolynomialFeatures(include_bias=True)),
  ('reg',LinearRegression())
  ])
  model = GridSearchCV(pipe,param_grid={'poly__degree':[1]})
  model.fit(x_train,y_train)
  degree = model.best_params_
  print(degree)
  pred = model.predict(x_test)
  MSE= metrics.mean_squared_error(y_test,pred)
  R2= metrics.r2_score(y_test,pred)
  print(f'MSE = {MSE:.3f}\nR2 Score = {R2:.3f}')

## F1

In [ ]:
X1 = data_f1.drop(columns=['Age','Gender','Status'])
make_model_classification(X1,'f1',data_f1.Gender)

Counter({0: 1422, 1: 1422})
>> The input dimension is 512 and it is reduced to 121
              precision    recall  f1-score   support

           0       0.98      0.98      0.98       484
           1       0.98      0.98      0.98       455

    accuracy                           0.98       939
   macro avg       0.98      0.98      0.98       939
weighted avg       0.98      0.98      0.98       939



In [ ]:
make_model_classification_knn(X1,'f1',data_f1.Gender)

Counter({0: 1422, 1: 1422})
>> The input dimension is 512 and it is reduced to 106
Best leaf_size: 1
Best p: 1
Best n_neighbors: 19
              precision    recall  f1-score   support

           0       0.97      0.97      0.97       484
           1       0.97      0.97      0.97       455

    accuracy                           0.97       939
   macro avg       0.97      0.97      0.97       939
weighted avg       0.97      0.97      0.97       939



In [ ]:
make_model_classification(X1,'f1',data_f1.Status)

Counter({0: 755, 2: 755, 1: 755})
>> The input dimension is 512 and it is reduced to 73
              precision    recall  f1-score   support

           0       0.76      0.76      0.76       231
           1       0.56      0.59      0.58       254
           2       0.67      0.64      0.65       263

    accuracy                           0.66       748
   macro avg       0.67      0.66      0.66       748
weighted avg       0.66      0.66      0.66       748



In [ ]:
make_model_classification_knn(X1,'f1',data_f1.Status)

Counter({0: 755, 2: 755, 1: 755})
>> The input dimension is 512 and it is reduced to 62
Best leaf_size: 2
Best p: 2
Best n_neighbors: 17
              precision    recall  f1-score   support

           0       0.70      0.77      0.73       231
           1       0.49      0.56      0.52       254
           2       0.64      0.49      0.56       263

    accuracy                           0.60       748
   macro avg       0.61      0.61      0.60       748
weighted avg       0.61      0.60      0.60       748



In [ ]:
make_model_regression(X1,'f1',data_f1.Age)

>> The input dimension is 512 and it is reduced to 81
{'poly__degree': 1}
MSE = 56.957
R2 Score = 0.737


In [ ]:
make_model_regression(X1,'f1',data_f1.Age)

>> The input dimension is 512 and it is reduced to 241
{'poly__degree': 1}
MSE = 37.660
R2 Score = 0.891


## F2

In [ ]:
X2 = data_f2.drop(columns=['Age','Gender','Status'])
make_model_classification(X2,'f2',data_f2.Gender)

Counter({0: 1445, 1: 1445})
>> The input dimension is 1536 and it is reduced to 75
              precision    recall  f1-score   support

           0       0.94      0.98      0.96       477
           1       0.98      0.94      0.96       477

    accuracy                           0.96       954
   macro avg       0.96      0.96      0.96       954
weighted avg       0.96      0.96      0.96       954



In [ ]:
make_model_classification_knn(X2,'f2',data_f2.Gender)


Counter({0: 1445, 1: 1445})
>> The input dimension is 1536 and it is reduced to 60
Best leaf_size: 1
Best p: 1
Best n_neighbors: 19
              precision    recall  f1-score   support

           0       0.93      0.99      0.96       477
           1       0.98      0.93      0.96       477

    accuracy                           0.96       954
   macro avg       0.96      0.96      0.96       954
weighted avg       0.96      0.96      0.96       954



In [ ]:
make_model_classification(X2,'f2',data_f2.Status)

Counter({0: 763, 1: 763, 2: 763})
>> The input dimension is 1536 and it is reduced to 42
              precision    recall  f1-score   support

           0       0.54      0.63      0.58       247
           1       0.46      0.28      0.35       272
           2       0.49      0.64      0.56       237

    accuracy                           0.51       756
   macro avg       0.50      0.51      0.49       756
weighted avg       0.50      0.51      0.49       756



In [ ]:
make_model_classification_knn(X2,'f2',data_f2.Status)

Counter({0: 763, 1: 763, 2: 763})
>> The input dimension is 1536 and it is reduced to 50
Best leaf_size: 3
Best p: 2
Best n_neighbors: 16
              precision    recall  f1-score   support

           0       0.55      0.66      0.60       247
           1       0.45      0.36      0.40       272
           2       0.51      0.52      0.51       237

    accuracy                           0.51       756
   macro avg       0.50      0.51      0.51       756
weighted avg       0.50      0.51      0.50       756



In [ ]:
make_model_regression(X2,'f2',data_f2.Age)

>> The input dimension is 1536 and it is reduced to 57
{'poly__degree': 1}
MSE = 45.531
R2 Score = 0.789


In [ ]:
make_model_regression(X2,'f2',data_f2.Age)

>> The input dimension is 1536 and it is reduced to 238
{'poly__degree': 1}
MSE = 31.373
R2 Score = 0.911


## F3

In [ ]:
X3 = data_f3.drop(columns=['Age','Gender','Status'])
make_model_classification(X3,'f3',data_f3.Gender)

Counter({0: 1445, 1: 1445})
>> The input dimension is 2304 and it is reduced to 80
              precision    recall  f1-score   support

           0       0.94      0.98      0.96       477
           1       0.98      0.94      0.96       477

    accuracy                           0.96       954
   macro avg       0.96      0.96      0.96       954
weighted avg       0.96      0.96      0.96       954



In [ ]:
make_model_classification_knn(X3,'f3',data_f3.Gender)

Counter({0: 1445, 1: 1445})
>> The input dimension is 2304 and it is reduced to 69
Best leaf_size: 1
Best p: 1
Best n_neighbors: 11
              precision    recall  f1-score   support

           0       0.94      0.98      0.96       477
           1       0.98      0.94      0.96       477

    accuracy                           0.96       954
   macro avg       0.96      0.96      0.96       954
weighted avg       0.96      0.96      0.96       954



In [ ]:
make_model_classification(X3,'f3',data_f3.Status)


Counter({0: 763, 1: 763, 2: 763})
>> The input dimension is 2304 and it is reduced to 32
              precision    recall  f1-score   support

           0       0.45      0.60      0.51       247
           1       0.43      0.19      0.27       272
           2       0.45      0.57      0.50       237

    accuracy                           0.44       756
   macro avg       0.44      0.45      0.43       756
weighted avg       0.44      0.44      0.42       756



In [ ]:
make_model_classification_knn(X3,'f3',data_f3.Status)


Counter({0: 763, 1: 763, 2: 763})
>> The input dimension is 2304 and it is reduced to 26
Best leaf_size: 1
Best p: 1
Best n_neighbors: 18
              precision    recall  f1-score   support

           0       0.44      0.60      0.51       247
           1       0.39      0.28      0.32       272
           2       0.44      0.43      0.44       237

    accuracy                           0.43       756
   macro avg       0.43      0.43      0.42       756
weighted avg       0.42      0.43      0.42       756



In [ ]:
make_model_regression(X3,'f3',data_f3.Age)

>> The input dimension is 2304 and it is reduced to 58
{'poly__degree': 1}
MSE = 50.655
R2 Score = 0.765


In [ ]:
make_model_regression(X3,'f3',data_f3.Age)

>> The input dimension is 2304 and it is reduced to 200
{'poly__degree': 1}
MSE = 35.131
R2 Score = 0.900


## F4

In [ ]:
X4 = data_f4.drop(columns=['Age','Gender','Status'])
make_model_classification(X4,'f4',data_f4.Gender)

Counter({0: 1643, 1: 1643})
>> The input dimension is 2560 and it is reduced to 72
              precision    recall  f1-score   support

           0       0.71      0.81      0.75       551
           1       0.77      0.66      0.71       534

    accuracy                           0.73      1085
   macro avg       0.74      0.73      0.73      1085
weighted avg       0.74      0.73      0.73      1085



In [ ]:
make_model_classification_knn(X4,'f4',data_f4.Gender)


Counter({0: 1643, 1: 1643})
>> The input dimension is 2560 and it is reduced to 57
Best leaf_size: 1
Best p: 1
Best n_neighbors: 18
              precision    recall  f1-score   support

           0       0.71      0.72      0.72       551
           1       0.71      0.69      0.70       534

    accuracy                           0.71      1085
   macro avg       0.71      0.71      0.71      1085
weighted avg       0.71      0.71      0.71      1085



In [ ]:
make_model_classification(X4,'f4',data_f4.Status)


Counter({0: 851, 2: 851, 1: 851})
>> The input dimension is 2560 and it is reduced to 2
              precision    recall  f1-score   support

           0       0.34      0.63      0.44       262
           1       0.38      0.23      0.28       287
           2       0.44      0.29      0.35       294

    accuracy                           0.37       843
   macro avg       0.39      0.38      0.36       843
weighted avg       0.39      0.37      0.36       843



In [ ]:
make_model_classification_knn(X4,'f4',data_f4.Status)


Counter({0: 851, 2: 851, 1: 851})
>> The input dimension is 2560 and it is reduced to 4
Best leaf_size: 1
Best p: 1
Best n_neighbors: 11
              precision    recall  f1-score   support

           0       0.32      0.43      0.37       262
           1       0.31      0.29      0.30       287
           2       0.40      0.31      0.35       294

    accuracy                           0.34       843
   macro avg       0.35      0.34      0.34       843
weighted avg       0.35      0.34      0.34       843



In [ ]:
make_model_regression(X4,'f4',data_f4.Age)

>> The input dimension is 2560 and it is reduced to 37
{'poly__degree': 1}
MSE = 196.282
R2 Score = 0.132


In [ ]:
make_model_regression(X4,'f4',data_f4.Age)

>> The input dimension is 2560 and it is reduced to 270
{'poly__degree': 1}
MSE = 141.794
R2 Score = 0.589


## F5

In [ ]:
X5 = data_f5.drop(columns=['Age','Gender','Status'])
make_model_classification(X5,'f5',data_f5.Gender)

Counter({0: 1115, 1: 1115})
>> The input dimension is 204 and it is reduced to 24
              precision    recall  f1-score   support

           0       0.77      0.67      0.72       379
           1       0.69      0.79      0.74       357

    accuracy                           0.73       736
   macro avg       0.73      0.73      0.73       736
weighted avg       0.73      0.73      0.73       736



In [ ]:
make_model_classification_knn(X5,'f5',data_f5.Gender)

Counter({0: 1115, 1: 1115})
>> The input dimension is 204 and it is reduced to 23
Best leaf_size: 1
Best p: 1
Best n_neighbors: 9
              precision    recall  f1-score   support

           0       0.74      0.72      0.73       379
           1       0.71      0.73      0.72       357

    accuracy                           0.72       736
   macro avg       0.72      0.72      0.72       736
weighted avg       0.72      0.72      0.72       736



In [ ]:
make_model_classification(X5,'f5',data_f5.Status)

Counter({0: 566, 2: 566, 1: 566})
>> The input dimension is 204 and it is reduced to 19
              precision    recall  f1-score   support

           0       0.63      0.68      0.66       201
           1       0.45      0.46      0.45       171
           2       0.55      0.50      0.52       189

    accuracy                           0.55       561
   macro avg       0.54      0.55      0.54       561
weighted avg       0.55      0.55      0.55       561



In [ ]:
make_model_classification_knn(X5,'f5',data_f5.Status)


Counter({0: 566, 2: 566, 1: 566})
>> The input dimension is 204 and it is reduced to 19
Best leaf_size: 2
Best p: 2
Best n_neighbors: 18
              precision    recall  f1-score   support

           0       0.60      0.65      0.62       201
           1       0.36      0.46      0.40       171
           2       0.51      0.34      0.41       189

    accuracy                           0.49       561
   macro avg       0.49      0.48      0.48       561
weighted avg       0.50      0.49      0.48       561



In [ ]:
make_model_regression(X5,'f5',data_f5.Age)

>> The input dimension is 204 and it is reduced to 24
{'poly__degree': 1}
MSE = 190.413
R2 Score = 0.063


In [ ]:
make_model_regression(X5,'f5',data_f5.Age)

>> The input dimension is 204 and it is reduced to 39
{'poly__degree': 1}
MSE = 238.696
R2 Score = 0.266
